In [6]:
import cv2
import xml.etree.ElementTree as ET
import os
from tqdm.notebook import tqdm

main_path = "../../"

In [44]:
# Generic
def draw_bbox(frame, bbox):
    for obj in bbox:
        x1, y1, x2, y2 = obj["bndbox"]
        obj_id = obj["id"]
        cls = obj["class"]

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {obj_id}, cls: {cls}", (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame

def video_builder(input_folder, output_path, name, fps = 30):
    files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]
    files.sort()

    first_frame_path = os.path.join(input_folder, files[0])
    frame = cv2.imread(first_frame_path)
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec per MP4
    video = cv2.VideoWriter(output_path + name, fourcc, fps, (width, height))

    for file in tqdm(files):
        frame_path = os.path.join(input_folder, file)
        frame = cv2.imread(frame_path)
        video.write(frame)

    video.release()

In [ ]:
# Per GRAM-RTM Dataset

def read_csv_data(path_annotations, frame_name):
    tree = ET.parse(os.path.join(path_annotations, frame_name + '.xml'))
    root = tree.getroot()
    bbox = []
    for child in root.iter('object'):
        obj = {}
        obj["class"] = child.find("class").text
        obj["id"] = child.find("ID").text

        bndbox = child.find('bndbox')

        x1 = bndbox.find("xmin").text
        y1 = bndbox.find("ymin").text
        x2 = bndbox.find("xmax").text
        y2 = bndbox.find("ymax").text
        #for subtag in bndbox:
        #    bndbox_data[subtag.tag] = subtag.text
        obj["bndbox"] = (x1, y1, x2, y2)
        
        bbox.append(obj)

    return bbox

def build_gram(input_folder, ds_annotations, output_file, fps = 30):
    # Importa cartella dei frames
    files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]
    files.sort()

    first_frame_path = os.path.join(input_folder, files[0])
    frame = cv2.imread(first_frame_path)
    height, width, layers = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec per MP4
    video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    for i, file in enumerate(tqdm(files)):
        frame_path = os.path.join(input_folder, file)
        frame = cv2.imread(frame_path)
        obj = read_csv_data(ds_annotations, str(i))

        video.write(draw_bbox(frame, obj))

    video.release()

In [ ]:
# Crea video che mostri dataset gram con le annotation
ds_name = "M-30-HD"
input_folder = f"{main_path}datasets/GRAM-RTMv4/Images/{ds_name}"
ds_annotations = f"{main_path}datasets/GRAM-RTMv4/Annotations/{ds_name}/xml/"
output_name = f"{main_path}datasets/GRAM-RTMv4/Videos/{ds_name}_with_annotations.mp4"
build_gram(input_folder, ds_annotations, output_name)

In [ ]:
# Crea video da frames
ds_name = "M-30"
input_folder = f"{main_path}datasets/GRAM-RTMv4/Images/{ds_name}"
output_path = f"{main_path}datasets/GRAM-RTMv4/Videos"
name = f"{ds_name}.mp4"
video_builder(input_folder, output_path, name, fps = 30)